In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import itertools

##### Fetch data

In [2]:
names = ['mobile','action_time','touchpoint_id','action_rank']
df = pd.read_csv('data/rw_tp_analysis_base.csv',sep = '\t', names = names)
df['mobile'] = df['mobile'].astype(str)

In [3]:
df['touchpoint_id'] = ['013002002000_rw' if i == '013004002000_rw' else i for i in df['touchpoint_id']]

In [4]:
df.head(3)

,mobile,action_time,touchpoint_id,action_rank
0,13001009310,2021-03-08 17:52:58,002005005001_rw,1
1,13001124343,2021-04-24 06:25:03,001008001001_rw,1
2,13001124343,2021-04-24 06:25:03,001001001000_rw,2


In [5]:
# id mapping
id_mapping = pd.read_csv('data/rw_id_mapping.csv')
tp_name = {k:v for k,v in zip(id_mapping.touchpoint_id,id_mapping.touchpoint_name)}

In [6]:
df = df.merge(id_mapping, on = 'touchpoint_id', how = 'left')

In [7]:
df = df[df.touchpoint_id.isna()==False]

- 提取关键节点

In [8]:
# RW deal date
deal_date = pd.read_csv('data/rw_deal_base.csv',sep = '\t', names = ['mobile','deal_time'])

/app/anaconda3/envs/python37_smcsandbox/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
deal_date = deal_date.dropna()
deal_date['deal_time'] = pd.to_datetime(deal_date['deal_time'])
min_deal_df = deal_date.groupby(by='mobile',as_index=False).agg({'deal_time':'min'})

In [20]:
# 意向前
oppor_df = df[df.touchpoint_id == '004000000000_rw']

min_oppor_df = oppor_df[['mobile','action_time']].groupby(by='mobile',as_index=False).agg({'action_time':'min'})
min_oppor_df.columns = ['mobile','oppor_time']

In [21]:
df_arrive_store = df[df.touchpoint_id=='006000000000_rw']
min_visit_df = df_arrive_store[['mobile','action_time']].groupby(by='mobile',as_index=False).agg({'action_time':'min'})\
.rename(columns={'action_time':'visit_time'})

In [22]:
register_df = df[df.touchpoint_id.str.startswith('002001')].groupby(by='mobile',as_index=False).agg({'action_time':'min'})
register_df.columns = ['mobile','register_time']

#### 1 - segment

In [9]:
# 意向 / 建卡
cust_user = df[(df.touchpoint_id == '004000000000_rw') | (df.touchpoint_id == '005000000000_rw')].mobile.unique().tolist()

In [12]:
old_deal = deal_date[deal_date.deal_time <= '2020-01-01']
new_deal = deal_date[deal_date.deal_time > '2020-01-01']

In [13]:
# segment
df['segment'] = np.nan
df['segment'][df.mobile.isin(old_deal.mobile.unique().tolist())] = 'deal' # 'old_deal'
df['segment'][df.mobile.isin(new_deal.mobile.unique().tolist())] = 'deal' # 'new_deal'
df['segment'][(df.mobile.isin(cust_user)) & (df.segment.isna()==True)] = 'cust'
df['segment'] = df['segment'].fillna('potential')

/app/anaconda3/envs/python37_smcsandbox/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
df[['mobile','segment']].drop_duplicates().segment.value_counts()

cust         1429483
potential     819071
deal          733546
Name: segment, dtype: int64

#### 一方触点

In [8]:
module_dic = {'内容':['002002003000_rw','002002003001_rw','002002003002_rw','002002003003_rw','002002003004_rw',
       '002002004000_rw','002002004001_rw','002002004002_rw','002002004003_rw','002002004004_rw',
       '002002004005_rw','002002004006_rw','002002004007_rw','002002006000_rw','002002006001_rw',
       '002002006002_rw','002002006003_rw','002002007000_rw','002002008000_rw','002002009000_rw',
       '002002010000_rw','002002005000_rw','002003003000_rw','002003003001_rw','002003003002_rw','002003004000_rw'],
 '消息':['002002012000_rw','002002012001_rw','002002012002_rw','002002012003_rw','002002012004_rw',
       '002002012005_rw','002002012006_rw','002002012007_rw','002002012008_rw','002002012009_rw',
       '002003002000_rw','002003002001_rw','002003002002_rw','002003002003_rw','002003002004_rw',
       '002003002005_rw','002003002006_rw','002003002007_rw'],
 '商城':['002002014000_rw','002002014001_rw'],
 '搜索':['002002011000_rw','002002011001_rw','002002011002_rw','002002011003_rw','002003001000_rw',
       '002003001001_rw','002003001002_rw'],
 '金融':['002004000000_rw','002004001000_rw','002004003000_rw','002004003001_rw','002004003002_rw','002004003003_rw'],
 '展厅':['003001000000_rw','003001001000_rw','003001001001_rw','003001001002_rw','003001001003_rw','003001001004_rw',
       '003001001005_rw','003001001006_rw','003001001007_rw','003001001008_rw','003001001009_rw','003001001010_rw',
       '003001001011_rw','003001001012_rw','003001002000_rw','003001002001_rw','003001002002_rw','003002000000_rw',
       '003002001000_rw','003002001001_rw','003002001002_rw','003002001003_rw','003002001004_rw','003002001005_rw',
       '003002001006_rw','003002001007_rw','003002001008_rw','003002002000_rw','003002002001_rw','003002002002_rw',
       '003002002003_rw','003002004000_rw'],
 '试驾':['007001001000_rw','007001001001_rw','007001001002_rw','007001001003_rw','007001001004_rw','007001001005_rw',
       '007001001006_rw','007001002000_rw','007001002001_rw','007001002002_rw'],
 '活动':['008002002000_rw','008002002001_rw','008002002002_rw','008002002003_rw','008002002004_rw','008002002005_rw','008002002006_rw',
     '008002002007_rw','008002002008_rw','008002002009_rw','008002002010_rw','008002002011_rw','008002003000_rw','008002003001_rw',
     '008002005000_rw','008002005001_rw'],
 '服务':['016001000000_rw','016001002000_rw','016001007000_rw','016001003000_rw','016001004000_rw','016001005000_rw','016001006000_rw']
}

In [9]:
app_id = []
key_id = []
for k in module_dic.keys():
    app_id += module_dic[k]
    len_ = len(module_dic[k])
    key_id += [k] * len_
    
module_map = {k:v for k,v in zip(app_id,key_id)}

In [10]:
app_df = df[df.touchpoint_id.isin(app_id)]
app_df['module'] = app_df['touchpoint_id'].apply(lambda x:module_map[x])

/app/anaconda3/envs/python37_smcsandbox/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# lead_source
lead_df = df[df.touchpoint_id.str.startswith('001') & (df.touchpoint_id.str.startswith('001001') == False)\
             & (df.touchpoint_id.str.startswith('001002') == False)]
lead_df_g = lead_df[['mobile','action_rank']].groupby(by='mobile',as_index=False).agg({'action_rank':'min'})
lead_df_g = lead_df_g.merge(lead_df,on=['mobile','action_rank'],how='inner')
lead_df_g['lead_type'] = ['汽车垂媒' if i.startswith('001008') else '非垂媒网销' for i in lead_df_g.touchpoint_id]

# cards & oppor
card_df = df[(df.touchpoint_id == '004000000000_rw') | (df.touchpoint_id == '005000000000_rw')][['mobile','action_time','touchpoint_id']]

In [ ]:
usr_base = pd.concat([lead_df_g[['mobile']],card_df[['mobile']]]).drop_duplicates()
usr_base = usr_base.merge(lead_df_g, on = 'mobile', how = 'left')
usr_base['lead_type'] = usr_base['lead_type'].fillna('线下')

In [ ]:
# all user
usr_base['lead_type'].value_counts()

In [ ]:
# dealed
usr_base.merge(min_deal_df,on='mobile',how='left').groupby(by='lead_type').agg({'deal_time':'count'})

In [ ]:
app_df = app_df.merge(usr_base[['mobile','lead_type']],on='mobile',how='left')

In [ ]:
app_df[['mobile','module','lead_type']].drop_duplicates().groupby(by=['lead_type','module'],as_index=False).agg({'mobile':'count'})\
.to_csv('data/rw_result/app_cov.csv',index=False)

In [ ]:
app_df[['mobile','lead_type']].drop_duplicates().groupby(by=['lead_type'],as_index=False).agg({'mobile':'count'})

- 售前、售后两大阶段，不同渠道来源的用户在APP各模块中的覆盖度

In [16]:
app_df = app_df.merge(min_deal_df,on='mobile',how='left')

In [20]:
app_df['deal_lag'] = (pd.to_datetime(app_df.action_time) - pd.to_datetime(app_df.deal_time)).apply(lambda x:x.days).fillna(-1)
app_df['deal_time_flag'] = ['after' if i >= 0 else 'before' for i in app_df['deal_lag']]
app_df['deal_flag'] = [1 if i <= 0 and i != -1 else 0 for i in app_df['deal_lag']]

In [ ]:
app_df[['mobile','module','lead_type','deal_time_flag']].drop_duplicates()\
.groupby(by=['lead_type','module','deal_time_flag'],as_index=False).agg({'mobile':'count'})\
.to_csv('data/rw_result/app_cov_by_stage.csv',index=False)

In [ ]:
app_df[['mobile','lead_type','deal_time_flag']].drop_duplicates()\
.groupby(by=['lead_type','deal_time_flag'],as_index=False).agg({'mobile':'count'})\

In [34]:
module_deal = app_df.groupby(by=['mobile','module'],as_index=False).agg({'deal_flag':'max'})

In [36]:
module_deal.groupby(by='module',as_index=False).agg({'deal_flag':'mean'})

,module,deal_flag
0,内容,0.038243
1,商城,0.020198
2,展厅,0.089768
3,搜索,0.034032
4,服务,0.054178
5,活动,0.005279
6,消息,0.033827
7,试驾,0.083165
8,金融,0.162663


In [39]:
app_df.groupby(by=['mobile'],as_index=False).agg({'deal_flag':'max'}).deal_flag.mean()

0.059855060771015954

- 不同内容模块在留资、到店、试驾三个阶段的覆盖度

In [ ]:
# 意向df
# df_make_order, df_arrive_store

In [ ]:
app_df = app_df.merge(min_oppor_df,on='mobile',how='left')
app_df['oppor_lag'] = (pd.to_datetime(app_df.action_time) - pd.to_datetime(app_df.oppor_time)).apply(lambda x:x.days)
app_df['oppor_time_flag'] = ['before_oppor' if i[0] <= 0 and i[1] <= 0 else 'others' for i in zip(app_df['deal_lag'],app_df['oppor_lag'])]

In [ ]:
app_df[['mobile','module','lead_type','oppor_time_flag']].drop_duplicates()\
.groupby(by=['lead_type','module','oppor_time_flag'],as_index=False).agg({'mobile':'count'})\
.to_csv('data/rw_result/app_cov_by_stage_bf_oppor.csv',index=False)

In [ ]:
app_df[['mobile','lead_type','oppor_time_flag']].drop_duplicates()\
.groupby(by=['lead_type','oppor_time_flag'],as_index=False).agg({'mobile':'count'})

In [ ]:
# 意向 - 到店 使用覆盖度
app_df = app_df.merge(min_visit_df,on='mobile',how='left')
app_df['visit_lag'] = (pd.to_datetime(app_df.action_time) - pd.to_datetime(app_df.visit_time)).apply(lambda x:x.days)
app_df['oppor_visit_flag'] = ['oppor_visit' if i[0] >= 0 and i[1] <= 0 else 'others' for i in zip(app_df['oppor_lag'],app_df['visit_lag'])]

In [ ]:
app_df[['mobile','module','lead_type','oppor_visit_flag']].drop_duplicates()\
.groupby(by=['lead_type','module','oppor_visit_flag'],as_index=False).agg({'mobile':'count'})\
.to_csv('data/rw_result/app_cov_by_stage_oppor_visit.csv',index=False)

In [ ]:
app_df[['mobile','lead_type','oppor_visit_flag']].drop_duplicates()\
.groupby(by=['lead_type','oppor_visit_flag'],as_index=False).agg({'mobile':'count'})

In [ ]:
# 到店 - 成交 使用覆盖度
app_df['visit_deal_flag'] = ['visit_deal' if i[0] <= 0 and i[0] != -1 and i[1] >= 0 else 'others' for i in zip(app_df['deal_lag'],app_df['visit_lag'])]

In [ ]:
app_df[['mobile','module','lead_type','visit_deal_flag']].drop_duplicates()\
.groupby(by=['lead_type','module','visit_deal_flag'],as_index=False).agg({'mobile':'count'})\
.to_csv('data/rw_result/app_cov_by_stage_visit_deal.csv',index=False)

In [ ]:
app_df[['mobile','lead_type','visit_deal_flag']].drop_duplicates()\
.groupby(by=['lead_type','visit_deal_flag'],as_index=False).agg({'mobile':'count'})

- 使用过APP各功能模块之后的7天活跃度

In [8]:
app_df['action_date'] = pd.to_datetime(app_df['action_time']).dt.date
app_date = app_df[['mobile','action_date','module']].drop_duplicates()
app_act = app_date[['mobile','action_date']].drop_duplicates()

/app/anaconda3/envs/python37_smcsandbox/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
for module in ['消息', '商城', '搜索', '金融', '展厅', '试驾', '活动', '服务']:
    print('Processing:',module)
    tmp_df = app_date[app_date.module == module].merge(app_act,how='left',on='mobile')
    tmp_df = tmp_df[tmp_df.action_date_x < tmp_df.action_date_y]
    tmp_df['date_diff'] = (tmp_df.action_date_y - tmp_df.action_date_x).apply(lambda x:x.days)
    print(tmp_df[['mobile','date_diff']].drop_duplicates().date_diff.value_counts()[:7])

Processing: 消息
1    63146
2    52506
3    48674
4    45203
5    42453
6    41621
7    40730
Name: date_diff, dtype: int64
Processing: 商城
1    22828
2    18828
3    17176
4    15929
5    15076
6    14758
7    13988
Name: date_diff, dtype: int64
Processing: 搜索
1    25184
2    19369
3    17642
4    16254
5    15137
6    14846
7    14426
Name: date_diff, dtype: int64
Processing: 金融
1    1482
2    1114
3    1006
4     915
5     819
6     791
7     734
Name: date_diff, dtype: int64
Processing: 展厅
1    41086
2    31981
3    28572
4    26515
5    24246
6    23669
7    23334
Name: date_diff, dtype: int64
Processing: 试驾
1    4722
2    3572
3    3275
4    2920
5    2739
6    2612
7    2554
Name: date_diff, dtype: int64
Processing: 活动
1    10652
2     8410
3     7374
4     7043
5     6429
6     6409
7     6061
Name: date_diff, dtype: int64
Processing: 服务
1    136918
2    109439
3     98638
4     91263
5     84386
6     82092
7     79858
Name: date_diff, dtype: int64
